In [1]:
import os
import pyspark
conf = pyspark.SparkConf()
conf.set('spark.ui.proxyBase', '/user/' + os.environ['JUPYTERHUB_USER'] + '/proxy/4041')
conf.set('spark.sql.repl.eagerEval.enabled', True)
conf.set('spark.driver.memory','4g')
sc = pyspark.SparkContext(conf=conf)
spark = pyspark.SQLContext.getOrCreate(sc)
sc.setLogLevel("ERROR")

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/10/12 20:19:12 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


/opt/conda/envs/bigdata-fall22/lib/python3.7/site-packages/pyspark/sql/context.py:159: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning,


In [2]:
BreadBasket = spark\
.read.option("header",True)\
.option("inferSchema",True)\
.csv("shared/hw2/BreadBasket_DMS.csv")

In [3]:
BreadBasket.show()

+-------------------+-------------------+-----------+-------------+
|               Date|               Time|Transaction|         Item|
+-------------------+-------------------+-----------+-------------+
|2016-10-30 00:00:00|2022-10-12 09:58:11|          1|        Bread|
|2016-10-30 00:00:00|2022-10-12 10:05:34|          2| Scandinavian|
|2016-10-30 00:00:00|2022-10-12 10:05:34|          2| Scandinavian|
|2016-10-30 00:00:00|2022-10-12 10:07:57|          3|Hot chocolate|
|2016-10-30 00:00:00|2022-10-12 10:07:57|          3|          Jam|
|2016-10-30 00:00:00|2022-10-12 10:07:57|          3|      Cookies|
|2016-10-30 00:00:00|2022-10-12 10:08:41|          4|       Muffin|
|2016-10-30 00:00:00|2022-10-12 10:13:03|          5|       Coffee|
|2016-10-30 00:00:00|2022-10-12 10:13:03|          5|       Pastry|
|2016-10-30 00:00:00|2022-10-12 10:13:03|          5|        Bread|
|2016-10-30 00:00:00|2022-10-12 10:16:55|          6|    Medialuna|
|2016-10-30 00:00:00|2022-10-12 10:16:55|       

In [4]:
BreadBasket.toPandas().tail()

,Date,Time,Transaction,Item
21288,2017-04-09,2022-10-12 14:32:58,9682,Coffee
21289,2017-04-09,2022-10-12 14:32:58,9682,Tea
21290,2017-04-09,2022-10-12 14:57:06,9683,Coffee
21291,2017-04-09,2022-10-12 14:57:06,9683,Pastry
21292,2017-04-09,2022-10-12 15:04:24,9684,Smoothies


In [5]:
from pyspark.sql.functions import count
from pyspark.sql.functions import col


In [6]:
BreadBasket.describe().show()

+-------+------------------+----------------+
|summary|       Transaction|            Item|
+-------+------------------+----------------+
|  count|             21293|           21293|
|   mean| 4951.990889024562|            null|
| stddev|2787.7583996983885|            null|
|    min|                 1|      Adjustment|
|    max|              9684|Victorian Sponge|
+-------+------------------+----------------+



In [7]:
from pyspark.sql.functions import date_format,to_timestamp
from pyspark.sql.functions import hour
new_BreadBasket = BreadBasket.select(col("Date"),col("Time"),col("Transaction"),col("Item"), hour(col("Time")).alias("Hour"), date_format(col("Time"),'a').alias("AM or PM"))

In [8]:
new_BreadBasket

Date,Time,Transaction,Item,Hour,AM or PM
2016-10-30 00:00:00,2022-10-12 09:58:11,1,Bread,9,AM
2016-10-30 00:00:00,2022-10-12 10:05:34,2,Scandinavian,10,AM
2016-10-30 00:00:00,2022-10-12 10:05:34,2,Scandinavian,10,AM
2016-10-30 00:00:00,2022-10-12 10:07:57,3,Hot chocolate,10,AM
2016-10-30 00:00:00,2022-10-12 10:07:57,3,Jam,10,AM
2016-10-30 00:00:00,2022-10-12 10:07:57,3,Cookies,10,AM
2016-10-30 00:00:00,2022-10-12 10:08:41,4,Muffin,10,AM
2016-10-30 00:00:00,2022-10-12 10:13:03,5,Coffee,10,AM
2016-10-30 00:00:00,2022-10-12 10:13:03,5,Pastry,10,AM
2016-10-30 00:00:00,2022-10-12 10:13:03,5,Bread,10,AM


In [9]:
from pyspark.sql.functions import concat,lit
new_BreadBasket = new_BreadBasket.select(col("Date"),col("Time"),col("Transaction"),col("Item"),col("Hour"),col("AM or PM"),concat(col("hour"),lit(' '),col("AM or PM")).alias("Hour with Am or PM"))

In [10]:
new_BreadBasket.show()

+-------------------+-------------------+-----------+-------------+----+--------+------------------+
|               Date|               Time|Transaction|         Item|Hour|AM or PM|Hour with Am or PM|
+-------------------+-------------------+-----------+-------------+----+--------+------------------+
|2016-10-30 00:00:00|2022-10-12 09:58:11|          1|        Bread|   9|      AM|              9 AM|
|2016-10-30 00:00:00|2022-10-12 10:05:34|          2| Scandinavian|  10|      AM|             10 AM|
|2016-10-30 00:00:00|2022-10-12 10:05:34|          2| Scandinavian|  10|      AM|             10 AM|
|2016-10-30 00:00:00|2022-10-12 10:07:57|          3|Hot chocolate|  10|      AM|             10 AM|
|2016-10-30 00:00:00|2022-10-12 10:07:57|          3|          Jam|  10|      AM|             10 AM|
|2016-10-30 00:00:00|2022-10-12 10:07:57|          3|      Cookies|  10|      AM|             10 AM|
|2016-10-30 00:00:00|2022-10-12 10:08:41|          4|       Muffin|  10|      AM|          

In [11]:
new_BreadBasket.filter('Hour >= 9 and Hour<23').groupby("Item").count().sort(col("count").desc()).show(5)

+------+-----+
|  Item|count|
+------+-----+
|Coffee| 5259|
| Bread| 3151|
|   Tea| 1414|
|  Cake| 1017|
|Pastry|  797|
+------+-----+
only showing top 5 rows



**These are the top 5 items bought (count) for the time period between 9:00AM inclusive and 11:00PM exclusive**

***Question 2***

In [12]:
Restaurants = spark.read\
.option("header",True)\
.option("delimiter",";")\
.option("inferSchema",True)\
.csv("shared/hw2/Restaurants_in_Durham_County_NC.csv")

In [13]:
Restaurants.toPandas()

,ID,Premise_Name,Premise_Address1,Premise_Address2,Premise_City,Premise_State,Premise_Zip,Premise_Phone,Hours_Of_Operation,Opening_Date,...,Sewage,Insp_Freq,Est_Group_Desc,Risk,Smoking_Allowed,Type_Description,Rpt_Area_Desc,Status,Transitional_Type_Desc,geolocation
0,56060,WEST 94TH ST PUB,4711 HOPE VALLEY RD,SUITE 6C,DURHAM,NC,27707,(919) 403-0025,None,1994-09-01,...,3 - Municipal/Community,4,Full-Service Restaurant,4.0,NO,1 - Restaurant,Food Service,ACTIVE,FOOD,"35.9207272, -78.9573299"
1,58123,BROOKDALE DURHAM IFS,4434 BEN FRANKLIN BLVD,None,DURHAM,NC,27704,(919) 479-9966,None,2003-10-15,...,3 - Municipal/Community,4,Nursing Home,4.0,NO,16 - Institutional Food Service,Food Service,ACTIVE,FOOD,"36.0467802, -78.8895483"
2,70266,SMOOTHIE KING,1125 W. NC HWY 54 SUITE 806,None,DURHAM,NC,27707,(919) 489-7300,None,2009-07-09,...,3 - Municipal/Community,2,Fast Food Restaurant,2.0,NO,1 - Restaurant,Food Service,ACTIVE,FOOD,"35.9182655, -78.9593263"
3,97837,HAMPTON INN & SUITES,1542 N GREGSON ST,None,DURHAM,NC,27701,(919) 688-8880,None,2012-01-09,...,3 - Municipal/Community,2,Full-Service Restaurant,2.0,NO,1 - Restaurant,Food Service,ACTIVE,FOOD,"36.0183378, -78.9060312"
4,60690,BETTER LIVING CONCEPTS OF DURHAM,909 GARCIA ST,None,DURHAM,NC,27704,(919) 477-5825,None,2008-06-02,...,3 - Municipal/Community,1,None,0.0,N/A,43 - Residential Care,Residential Care,ACTIVE,N/A,"36.0556347, -78.9135175"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2467,59033,INDIGO APARTMENTS,100 ADELAIDE CIRCLE,None,MORRISVILLE,NC,27560,None,None,2002-04-29,...,3 - Municipal/Community,1,None,0.0,N/A,50 - Seasonal Swimming Pool,Swimming Pools,ACTIVE,N/A,"35.9236035, -78.8109587"
2468,59005,GREENS OF PINE GLEN,6201 PINE GLEN TR,None,DURHAM,NC,27713,None,None,2001-08-17,...,3 - Municipal/Community,1,None,0.0,N/A,50 - Seasonal Swimming Pool,Swimming Pools,ACTIVE,N/A,"35.907862, -78.920936"
2469,59104,RESIDENCE INN,201 RESIDENCE INN BLVD,None,DURHAM,NC,27713,None,None,1995-04-20,...,3 - Municipal/Community,1,None,0.0,N/A,52 - Seasonal Spa,Swimming Pools,ACTIVE,N/A,"35.9041757, -78.8974251"
2470,59106,SHERATON IMPERIAL,4700 EMPEROR BLVD,None,RTP,NC,27709,None,None,1994-07-18,...,3 - Municipal/Community,1,None,0.0,N/A,52 - Seasonal Spa,Swimming Pools,ACTIVE,N/A,"35.8779479, -78.8384012"


In [14]:
Grouped_restaurants= Restaurants.groupBy("Rpt_Area_Desc")

In [15]:
Grouped_restaurants.count().toPandas()

,Rpt_Area_Desc,count
0,Bed&Breakfast Home,4
1,Summer Camps,4
2,Institutions,30
3,Local Confinement,2
4,Mobile Food,147
5,School Buildings,89
6,None,13
7,Summer Food,242
8,Swimming Pools,420
9,Day Care,173


***Question 3***

In [16]:
Population=  spark.read\
.option("header",True)\
.option("inferSchema",True)\
.csv("shared/hw2/populationbycountry19802010millions.csv")

In [17]:
Population= Population.withColumnRenamed("_c0","Region")

In [18]:
Population.toPandas()

,Region,1980,1981,1982,1983,1984,1985,1986,1987,1988,...,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010
0,North America,320.27638,324.44694,328.62014,332.72487,336.72143,340.74811,344.89548,349.07829,353.2939,...,417.83236,422.05268,426.06238,430.26938,434.47232,438.82964,443.3473,447.67394,451.83698,456.59331
1,Bermuda,0.05473,0.05491,0.05517,0.05551,0.05585,0.05618,0.05651,0.05683,0.05717,...,0.06361,0.06418,0.06476,0.06534,0.06591,0.06644,0.06692,0.06739,0.06784,0.06827
2,Canada,24.5933,24.9,25.2019,25.4563,25.7018,25.9416,26.2038,26.5497,26.8948,...,31.37674,31.64096,31.88931,32.13476,32.38638,32.65668,32.93596,33.2127,33.48721,33.75974
3,Greenland,0.05021,0.05103,0.05166,0.05211,0.05263,0.05315,0.05364,0.0541,0.05485,...,0.05713,0.05736,0.05754,0.0577,0.05778,0.05764,0.05753,0.05756,0.0576,0.05764
4,Mexico,68.34748,69.96926,71.6409,73.36288,75.08014,76.76723,78.44243,80.12249,81.78182,...,101.24696,102.47993,103.71806,104.95959,106.2029,107.44953,108.70089,109.9554,111.21179,112.46886
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
227,U.S. Pacific Islands,0.13796,0.14283,0.14762,0.15281,0.15825,0.16406,0.17104,0.17903,0.18841,...,0.25365,0.25739,0.26116,0.26494,0.25814,0.24957,0.24888,0.24678,0.24424,0.24221
228,Vanuatu,0.11679,0.12026,0.12379,0.12738,0.13103,0.13473,0.13848,0.14227,0.14612,...,0.19317,0.19646,0.19971,0.20293,0.20609,0.20922,0.21234,0.21545,0.21852,0.22155
229,Vietnam,53.7152,54.90268,56.14218,57.43635,58.76204,60.09307,61.43983,62.82649,64.21135,...,80.20948,81.25796,82.29659,83.35245,84.42493,85.47054,86.51885,87.55836,88.57676,89.57113
230,Wake Island,NA,NA,NA,NA,NA,NA,NA,NA,NA,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA


In [19]:
sc.setLogLevel("ERROR")
from pyspark.sql.functions import lit
from pyspark.sql.functions import col

for year in range(1980, 2010):
    y2 = str(year+1)
    y1 = str(year)
    # add a column with the percentage change; drop the unneeded column after
    Population = Population.withColumn("prcnt_"+y2, lit(100.0)*(col(y2)-col(y1))/col(y1)).drop(y1)
Population = Population.drop("2010")

In [20]:
Population.limit(10).toPandas()

,Region,prcnt_1981,prcnt_1982,prcnt_1983,prcnt_1984,prcnt_1985,prcnt_1986,prcnt_1987,prcnt_1988,prcnt_1989,...,prcnt_2001,prcnt_2002,prcnt_2003,prcnt_2004,prcnt_2005,prcnt_2006,prcnt_2007,prcnt_2008,prcnt_2009,prcnt_2010
0,North America,1.302175,1.286250,1.249080,1.201161,1.195849,1.217137,1.212776,1.207640,1.242781,...,1.090635,1.010051,0.950047,0.987414,0.976816,1.002899,1.029479,0.975903,0.929927,1.052665
1,Bermuda,0.328887,0.473502,0.616277,0.612502,0.590868,0.587398,0.566271,0.598276,0.559734,...,0.872185,0.896086,0.903708,0.895615,0.872360,0.804127,0.722456,0.702331,0.667755,0.633844
2,Canada,1.247088,1.212450,1.009448,0.964398,0.933009,1.010732,1.320038,1.299826,1.801463,...,0.891267,0.842089,0.784900,0.769694,0.783015,0.834610,0.855200,0.840237,0.826521,0.813833
3,Greenland,1.633141,1.234568,0.871080,0.997889,0.988030,0.921919,0.857569,1.386322,1.020966,...,0.421867,0.402591,0.313808,0.278067,0.138648,-0.242298,-0.190840,0.052147,0.069493,0.069444
4,Mexico,2.372845,2.389106,2.403627,2.340775,2.247052,2.182181,2.141775,2.070992,1.938108,...,1.321310,1.217785,1.208168,1.197024,1.184561,1.173819,1.164603,1.154094,1.142636,1.130339
5,Saint Pierre and Miquelon,0.333890,0.665557,0.330579,0.658979,0.818331,0.811688,0.644122,0.480000,0.477707,...,-0.624025,-0.627943,-0.631912,-0.635930,-0.800000,-0.806452,-0.813008,-0.819672,-0.826446,-1.000000
6,United States,0.986262,0.958204,0.918367,0.869538,0.890025,0.928486,0.897849,0.912159,0.948981,...,1.031144,0.954937,0.876468,0.936642,0.923886,0.960281,1.000254,0.926769,0.864625,1.050893
7,Central & South America,2.174231,2.178203,2.143741,2.037747,1.990560,2.028082,2.041957,2.022114,1.956682,...,1.372919,1.360134,1.369030,1.374890,1.357210,1.321779,1.285418,1.263179,1.237416,1.153395
8,Antarctica,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,Antigua and Barbuda,-0.423049,-0.366247,-3.514189,-1.752514,-0.108578,0.000000,0.031056,0.015523,-0.062083,...,1.831453,1.576958,1.449833,1.416466,1.359272,1.304134,1.323779,1.306484,1.313299,1.307953


In [21]:
Population = Population.dropna()

*dropping the null values*

In [22]:
from pyspark.sql.functions import desc,asc
result1 = []
result2 = []
for year in range(1981, 2011):
    y = "prcnt_"+str(year)
    row1 = Population.select(col("Region"),col(y)).orderBy(desc(y)).limit(1).take(1)[0]
    row2 = Population.select(col("Region"),col(y)).orderBy(asc(y)).limit(1).take(1)[0]
    result1.append([year, row1.Region, row1[1]])
    result2.append([year, row2.Region, row2[1]])

In [23]:
result1

[[1981, 'Western Sahara', 12.133182844243787],
 [1982, 'Western Sahara', 11.115105327485804],
 [1983, 'French Guiana', 14.285714285714278],
 [1984, 'Qatar', 10.964057316781224],
 [1985, 'French Guiana', 12.499999999999995],
 [1986, 'Qatar', 8.771732719152874],
 [1987, 'French Guiana', 11.111111111111121],
 [1988, 'Cayman Islands', 11.010421386497516],
 [1989, 'United Arab Emirates', 6.119858265290403],
 [1990, 'Djibouti', 12.82404791501865],
 [1991, 'Jordan', 11.273939557210026],
 [1992, 'Kuwait', 48.63343882962002],
 [1993, 'Afghanistan', 13.224594754698687],
 [1994, 'Afghanistan', 8.727661664211226],
 [1995, 'Burundi', 7.222488903730302],
 [1996, 'Rwanda', 19.61417728550077],
 [1997, 'Falkland Islands (Islas Malvinas)', 21.49999999999999],
 [1998, 'Liberia', 12.01744976042338],
 [1999, 'Falkland Islands (Islas Malvinas)', 7.692307692307697],
 [2000, 'Montserrat', 16.863905325443792],
 [2001, 'Montserrat', 7.34177215189872],
 [2002, 'Montserrat', 13.4433962264151],
 [2003, 'Afghanista

In [44]:
top_countries = spark.createDataFrame(result1).toDF("Year","Top Country","Percent Change").toPandas()

In [24]:
result2

[[1981, 'Afghanistan', -9.106330931425992],
 [1982, 'Afghanistan', -8.017227257036874],
 [1983, 'Antigua and Barbuda', -3.5141890898397343],
 [1984, 'Antigua and Barbuda', -1.7525144772935055],
 [1985, 'Cook Islands', -1.4092446448703508],
 [1986, 'Netherlands Antilles', -24.58781655279631],
 [1987, 'Saint Helena', -21.299638989169676],
 [1988, 'Mozambique', -2.883631837516533],
 [1989, 'Somalia', -2.1964965331028314],
 [1990, 'Liberia', -12.816300240117076],
 [1991, 'Kuwait', -55.4531619095637],
 [1992, 'Somalia', -5.387440289087448],
 [1993, 'Bhutan', -4.150184489578821],
 [1994, 'Rwanda', -14.363511428676736],
 [1995, 'Rwanda', -15.871881307134093],
 [1996, 'Montserrat', -22.590068159688407],
 [1997, 'Montserrat', -25.157232704402517],
 [1998, 'Montserrat', -43.193277310924366],
 [1999, 'Cook Islands', -2.9919447640966608],
 [2000, 'Cook Islands', -3.2621589561091247],
 [2001, 'Cook Islands', -3.55610055180871],
 [2002, 'Cook Islands', -3.6872218690400547],
 [2003, 'Montserrat', -6.

In [45]:
bottom_countries = spark.createDataFrame(result1).toDF("Year","Bottom Country","Percent Change").toPandas()

**For each year, top country in terms of global growth:**

In [57]:
top_countries

,Year,Top Country,Percent Change
0,1981,Western Sahara,12.133183
1,1982,Western Sahara,11.115105
2,1983,French Guiana,14.285714
3,1984,Qatar,10.964057
4,1985,French Guiana,12.500000
5,1986,Qatar,8.771733
6,1987,French Guiana,11.111111
7,1988,Cayman Islands,11.010421
8,1989,United Arab Emirates,6.119858
9,1990,Djibouti,12.824048


**For each year, bottom country in terms of global growth**

In [58]:
bottom_countries

,Year,Bottom Country,Percent Change
0,1981,Western Sahara,12.133183
1,1982,Western Sahara,11.115105
2,1983,French Guiana,14.285714
3,1984,Qatar,10.964057
4,1985,French Guiana,12.500000
5,1986,Qatar,8.771733
6,1987,French Guiana,11.111111
7,1988,Cayman Islands,11.010421
8,1989,United Arab Emirates,6.119858
9,1990,Djibouti,12.824048


***Question 4***

In [25]:
romeo_juliet = spark.read\
.text("shared/hw2/romeo-juliet-pg1777.txt")

In [26]:
romeo_juliet.show()

+--------------------+
|               value|
+--------------------+
|                    |
|This Etext file i...|
|cooperation with ...|
|Future and Shakes...|
|Etexts that are N...|
|                    |
|*This Etext has c...|
|                    |
|<<THIS ELECTRONIC...|
|SHAKESPEARE IS CO...|
|PROVIDED BY PROJE...|
|MACHINE READABLE ...|
|(1) ARE FOR YOUR ...|
|DISTRIBUTED OR US...|
|DISTRIBUTION INCL...|
|TIME OR FOR MEMBE...|
|                    |
|*Project Gutenber...|
|in the presentati...|
|for your reading ...|
+--------------------+
only showing top 20 rows



In [27]:
from pyspark.sql import functions as F
romeo_juliet = romeo_juliet.select(F.regexp_replace(F.lower(F.col("value")),r'[^0-9a-zA-Z]',' ').alias("new_line"))

In [28]:
romeo_juliet.show()

+--------------------+
|            new_line|
+--------------------+
|                    |
|this etext file i...|
|cooperation with ...|
|future and shakes...|
|etexts that are n...|
|                    |
| this etext has c...|
|                    |
|  this electronic...|
|shakespeare is co...|
|provided by proje...|
|machine readable ...|
| 1  are for your ...|
|distributed or us...|
|distribution incl...|
|time or for membe...|
|                    |
| project gutenber...|
|in the presentati...|
|for your reading ...|
+--------------------+
only showing top 20 rows



In [29]:
romeo_juliet = romeo_juliet.select(F.explode(F.split("new_line",r'[ ]+')))

In [30]:
romeo_juliet = romeo_juliet.withColumnRenamed("col","words")

In [31]:
romeo_juliet.groupBy("words").count().toPandas()

,words,count
0,those,17
1,carnegie,10
2,some,58
3,chor,2
4,art,55
...,...,...
3907,surgeon,1
3908,living,5
3909,banishment,8
3910,infold,1


***Extra credit***

In [32]:
restaurants = spark.read\
    .option("header",True)\
    .option("delimiter",";")\
    .option("inferSchema",True)\
    .csv("shared/hw2/Restaurants_in_Durham_County_NC.csv")
    
restaurants.printSchema()
restaurants.limit(5).toPandas()

root
 |-- ID: string (nullable = true)
 |-- Premise_Name: string (nullable = true)
 |-- Premise_Address1: string (nullable = true)
 |-- Premise_Address2: string (nullable = true)
 |-- Premise_City: string (nullable = true)
 |-- Premise_State: string (nullable = true)
 |-- Premise_Zip: string (nullable = true)
 |-- Premise_Phone: string (nullable = true)
 |-- Hours_Of_Operation: string (nullable = true)
 |-- Opening_Date: string (nullable = true)
 |-- Closing_Date: string (nullable = true)
 |-- Seats: string (nullable = true)
 |-- Water: string (nullable = true)
 |-- Sewage: string (nullable = true)
 |-- Insp_Freq: string (nullable = true)
 |-- Est_Group_Desc: string (nullable = true)
 |-- Risk: integer (nullable = true)
 |-- Smoking_Allowed: string (nullable = true)
 |-- Type_Description: string (nullable = true)
 |-- Rpt_Area_Desc: string (nullable = true)
 |-- Status: string (nullable = true)
 |-- Transitional_Type_Desc: string (nullable = true)
 |-- geolocation: string (nullable = t

,ID,Premise_Name,Premise_Address1,Premise_Address2,Premise_City,Premise_State,Premise_Zip,Premise_Phone,Hours_Of_Operation,Opening_Date,...,Sewage,Insp_Freq,Est_Group_Desc,Risk,Smoking_Allowed,Type_Description,Rpt_Area_Desc,Status,Transitional_Type_Desc,geolocation
0,56060,WEST 94TH ST PUB,4711 HOPE VALLEY RD,SUITE 6C,DURHAM,NC,27707,(919) 403-0025,None,1994-09-01,...,3 - Municipal/Community,4,Full-Service Restaurant,4,NO,1 - Restaurant,Food Service,ACTIVE,FOOD,"35.9207272, -78.9573299"
1,58123,BROOKDALE DURHAM IFS,4434 BEN FRANKLIN BLVD,None,DURHAM,NC,27704,(919) 479-9966,None,2003-10-15,...,3 - Municipal/Community,4,Nursing Home,4,NO,16 - Institutional Food Service,Food Service,ACTIVE,FOOD,"36.0467802, -78.8895483"
2,70266,SMOOTHIE KING,1125 W. NC HWY 54 SUITE 806,None,DURHAM,NC,27707,(919) 489-7300,None,2009-07-09,...,3 - Municipal/Community,2,Fast Food Restaurant,2,NO,1 - Restaurant,Food Service,ACTIVE,FOOD,"35.9182655, -78.9593263"
3,97837,HAMPTON INN & SUITES,1542 N GREGSON ST,None,DURHAM,NC,27701,(919) 688-8880,None,2012-01-09,...,3 - Municipal/Community,2,Full-Service Restaurant,2,NO,1 - Restaurant,Food Service,ACTIVE,FOOD,"36.0183378, -78.9060312"
4,60690,BETTER LIVING CONCEPTS OF DURHAM,909 GARCIA ST,None,DURHAM,NC,27704,(919) 477-5825,None,2008-06-02,...,3 - Municipal/Community,1,None,0,N/A,43 - Residential Care,Residential Care,ACTIVE,N/A,"36.0556347, -78.9135175"


In [33]:
# look only at qualifying restaurants
interesting = restaurants.where('Status = "ACTIVE" AND Rpt_Area_Desc = "Food Service"')\
    .select("Premise_Name", "geolocation")\
    .withColumn("point", F.split("geolocation", ","))\
    .withColumn("lat", F.col("point")[0].cast("Double"))\
    .withColumn("lon", F.col("point")[1].cast("Double"))\
    .withColumn("coordinates", F.array(F.col("lat"), F.col("lon")))

interesting.limit(5).toPandas()

,Premise_Name,geolocation,point,lat,lon,coordinates
0,WEST 94TH ST PUB,"35.9207272, -78.9573299","[35.9207272, -78.9573299]",35.920727,-78.957330,"[35.9207272, -78.9573299]"
1,BROOKDALE DURHAM IFS,"36.0467802, -78.8895483","[36.0467802, -78.8895483]",36.046780,-78.889548,"[36.0467802, -78.8895483]"
2,SMOOTHIE KING,"35.9182655, -78.9593263","[35.9182655, -78.9593263]",35.918265,-78.959326,"[35.9182655, -78.9593263]"
3,HAMPTON INN & SUITES,"36.0183378, -78.9060312","[36.0183378, -78.9060312]",36.018338,-78.906031,"[36.0183378, -78.9060312]"
4,KROGER R 381 MEAT MARKET,"35.9495321, -78.9211694","[35.9495321, -78.9211694]",35.949532,-78.921169,"[35.9495321, -78.9211694]"


In [34]:
foreclosures = spark.read.json("shared/hw2/durham-nc-foreclosure-2006-2016.json")
foreclosures.limit(5).toPandas()

,datasetid,fields,geometry,record_timestamp,recordid
0,foreclosure-2006-2016,"(217 E CORPORATION ST, [36.0013755, -78.892254...","([-78.8922549, 36.0013755], Point)",2017-03-06T12:41:48-05:00,629979c85b1cc68c1d4ee8cc351050bfe3592c62
1,foreclosure-2006-2016,"(401 N QUEEN ST, [35.995797, -78.895396], 1105...","([-78.895396, 35.995797], Point)",2017-03-06T12:41:48-05:00,e3cce8bbc3c9b804cbd87e267a6ff121285274e0
2,foreclosure-2006-2016,"(403 N QUEEN ST, [35.995413, -78.8950321], 110...","([-78.8950321, 35.995413], Point)",2017-03-06T12:41:48-05:00,311559ebfeffe7ebc2a8b056a034a24298da08f3
3,foreclosure-2006-2016,"(918 GILBERT ST, [35.9957683, -78.8873774], 11...","([-78.8873774, 35.9957683], Point)",2017-03-06T12:41:48-05:00,7ec0761bd385bab8af10f682115a6eb4400740b3
4,foreclosure-2006-2016,"(721 LIBERTY ST, [35.993026, -78.888343], 1113...","([-78.888343, 35.993026], Point)",2017-03-06T12:41:48-05:00,c81ae2921ffca8125c2de2fd3e3b1375388cd925


In [35]:
#  get the foreclosure data we want
df1 = foreclosures.select("recordid","fields.geocode")
df1.limit(5).toPandas()

,recordid,geocode
0,629979c85b1cc68c1d4ee8cc351050bfe3592c62,"[36.0013755, -78.8922549]"
1,e3cce8bbc3c9b804cbd87e267a6ff121285274e0,"[35.995797, -78.895396]"
2,311559ebfeffe7ebc2a8b056a034a24298da08f3,"[35.995413, -78.8950321]"
3,7ec0761bd385bab8af10f682115a6eb4400740b3,"[35.9957683, -78.8873774]"
4,c81ae2921ffca8125c2de2fd3e3b1375388cd925,"[35.993026, -78.888343]"


In [36]:
df = interesting.select("Premise_Name", "coordinates")\
    .crossJoin(df1)

df.limit(5).toPandas()

,Premise_Name,coordinates,recordid,geocode
0,WEST 94TH ST PUB,"[35.9207272, -78.9573299]",629979c85b1cc68c1d4ee8cc351050bfe3592c62,"[36.0013755, -78.8922549]"
1,BROOKDALE DURHAM IFS,"[36.0467802, -78.8895483]",629979c85b1cc68c1d4ee8cc351050bfe3592c62,"[36.0013755, -78.8922549]"
2,SMOOTHIE KING,"[35.9182655, -78.9593263]",629979c85b1cc68c1d4ee8cc351050bfe3592c62,"[36.0013755, -78.8922549]"
3,HAMPTON INN & SUITES,"[36.0183378, -78.9060312]",629979c85b1cc68c1d4ee8cc351050bfe3592c62,"[36.0013755, -78.8922549]"
4,KROGER R 381 MEAT MARKET,"[35.9495321, -78.9211694]",629979c85b1cc68c1d4ee8cc351050bfe3592c62,"[36.0013755, -78.8922549]"


In [37]:
import haversine as H
import sys

def distance(p1, p2, unit):
    # check for missing values
    if p1 == None or p2 == None or p1[0] == None or p1[1] == None or p2[0] == None or p2[1] == None:
        return sys.maxsize
    return H.haversine(p1, p2, unit=unit)

In [38]:
udf_distance = F.udf(distance)

result = df.withColumn("distance", udf_distance(F.col("coordinates"), F.col("geocode"), lit('mi')).cast("Double"))
result.limit(10).toPandas()

,Premise_Name,coordinates,recordid,geocode,distance
0,WEST 94TH ST PUB,"[35.9207272, -78.9573299]",629979c85b1cc68c1d4ee8cc351050bfe3592c62,"[36.0013755, -78.8922549]",6.655446
1,BROOKDALE DURHAM IFS,"[36.0467802, -78.8895483]",629979c85b1cc68c1d4ee8cc351050bfe3592c62,"[36.0013755, -78.8922549]",3.140810
2,SMOOTHIE KING,"[35.9182655, -78.9593263]",629979c85b1cc68c1d4ee8cc351050bfe3592c62,"[36.0013755, -78.8922549]",6.858936
3,HAMPTON INN & SUITES,"[36.0183378, -78.9060312]",629979c85b1cc68c1d4ee8cc351050bfe3592c62,"[36.0013755, -78.8922549]",1.402282
4,KROGER R 381 MEAT MARKET,"[35.9495321, -78.9211694]",629979c85b1cc68c1d4ee8cc351050bfe3592c62,"[36.0013755, -78.8922549]",3.930000
5,LEONE INTERNATIONAL GROCERY/MEAT MARKET,"[35.9852771, -78.898944]",629979c85b1cc68c1d4ee8cc351050bfe3592c62,"[36.0013755, -78.8922549]",1.173467
6,PIZZA HUT DELIVERY,"[36.0586094, -78.9279599]",629979c85b1cc68c1d4ee8cc351050bfe3592c62,"[36.0013755, -78.8922549]",4.429251
7,CHUBBYS TACOS,"[36.0094173, -78.9218729]",629979c85b1cc68c1d4ee8cc351050bfe3592c62,"[36.0013755, -78.8922549]",1.746224
8,SUBWAY,"[36.018111, -78.9102149]",629979c85b1cc68c1d4ee8cc351050bfe3592c62,"[36.0013755, -78.8922549]",1.531233
9,DUKE UNIVERSITY WEST UNION FARMSTEAD,"[36.001517, -78.9391734]",629979c85b1cc68c1d4ee8cc351050bfe3592c62,"[36.0013755, -78.8922549]",2.622609


In [39]:
limit = 10.0

In [40]:
print(type(limit))

<class 'float'>


In [41]:
result.select("Premise_name", "distance")\
    .where(F.col("distance") <= F.lit(limit).cast("Double"))\
    .groupBy("Premise_Name").count()

Premise_Name,count
GRILL 46,1790
EL DORADO'S #6,1710
DUKE UNIVERSITY W...,1905
COMPARE FOODS DELI,1898
BLU SEAFOOD AND BAR,1901
MCDONALD'S 35265,1840
GSK COMMERCIAL OP...,1821
JADE BUFFET,1895
W G PEARSON SCHOO...,1891
DPAC 3RD FLOOR,1912
